In [ ]:
import sys
sys.path.append('..')
from pcutils.core.config import load_config
from pcutils.core.workspace import Workspace
from pcutils.plot.histogram import Histogrammer
import matplotlib.pyplot as plt
import polars as pl
import numpy as np
import math
from scipy import constants

QBRHO_2_P: float = 1.0e-9 * constants.speed_of_light * 100.0 * 10.0 #T * m -> MeV/c

In [ ]:
config = load_config('../local_config.json')
ws = Workspace(config.workspace)
nuclear_map = ws.get_nuclear_map()
ejectile = nuclear_map.get_data(1, 2)
projectile = nuclear_map.get_data(6, 16)
target = nuclear_map.get_data(1, 2)
residual = nuclear_map.get_data(6, 16)
proj_energy = 11.5 * projectile.atomic_mass #MeV
proj_momentum = np.sqrt(proj_energy * (proj_energy + 2.0 * projectile.mass))


In [ ]:

grammer = Histogrammer()
grammer.add_hist2d('ke_theta', (180, 1600), ((0.0, 90.0), (0.0, 50.0)))
grammer.add_hist2d('ke_phi', (360, 1000), ((0.0, 360.0), (0.0, 50.0)))
grammer.add_hist1d('ex', 1000, (-10.0, 10.0))

In [ ]:
proj_vec = np.array([0., 0., proj_momentum, proj_energy + projectile.mass])
targ_vec = np.array([0., 0., 0., target.mass])
for run in range(config.run.run_min, config.run.run_max+1):
    df = None
    try:
        df = pl.read_parquet(ws.get_physics_file_path_parquet(run, ejectile))
    except Exception:
        continue

    print(ejectile.Z)

    brho = df.select('brho').to_numpy().flatten()
    print(brho)
    momentum = df.select('brho').to_numpy().flatten() * float(ejectile.Z) * QBRHO_2_P
    eject_vecs = np.zeros((len(momentum), 4))
    kinetic_energy = np.sqrt(momentum**2.0 + ejectile.mass**2.0) - ejectile.mass
    print(kinetic_energy)
    polar = np.rad2deg(df.select('polar').to_numpy().flatten())
    az = np.rad2deg(df.select('azimuthal').to_numpy().flatten())
    eject_vecs[:, 0] = (np.sin(polar) * np.cos(az) * momentum)
    eject_vecs[:, 1] = (np.sin(polar) * np.sin(az) * momentum)
    eject_vecs[:, 2] = (np.cos(polar) * momentum)
    eject_vecs[:, 3] = (kinetic_energy + ejectile.mass)

    resid_vecs = proj_vec + targ_vec - eject_vecs

    exs = np.sqrt(resid_vecs[:, 3] ** 2.0 - np.linalg.norm(resid_vecs[:, :3], axis=1)**2.0) - residual.mass
    print(exs)

    grammer.fill_hist2d('ke_theta', polar, kinetic_energy)
    grammer.fill_hist2d('ke_phi', az, kinetic_energy)
    grammer.fill_hist1d('ex', exs)

In [ ]:
angle_range = np.linspace(0., np.pi*0.5, 1000)

ex = 0.0

Q = projectile.mass + target.mass - (ejectile.mass + residual.mass + ex)
term1 = np.sqrt(projectile.mass * ejectile.mass * proj_energy) / (ejectile.mass + residual.mass) * np.cos(angle_range)
term2 = (proj_energy * (residual.mass - projectile.mass) + residual.mass*Q) / (residual.mass + ejectile.mass)
eject_energy = term1 + np.sqrt(term1*term1 + term2)
eject_energy = eject_energy**2.0

In [ ]:
fig, ax = plt.subplots(1,2)
fig.set_figwidth(16)
fig.set_figheight(10)
mesh = grammer.draw_hist2d('ke_theta', ax[0], log_z=False)
ax[0].scatter(angle_range*180.0/np.pi, eject_energy, s=3, color='r')
ax[0].set_xlabel(r'$\theta_{Lab}$')
ax[0].set_ylabel('Kinetic Energy (MeV)')
ax[0].set_ylim(0.0, 40.0)
plt.colorbar(mesh, ax=ax[0])
mesh2 = grammer.draw_hist2d('ke_phi', ax[1], log_z=False)
ax[1].set_xlabel(r'$\phi_{Lab}$')
ax[1].set_ylabel('Kinetic Energy (MeV)')
plt.colorbar(mesh, ax=ax[1])
plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(1,1)
fig.set_figwidth(16)
fig.set_figheight(10)
grammer.draw_hist1d('ex', ax)